In [567]:
import numpy as np
from collections import Counter
import random

In [425]:
sudoku_mat = np.array([
              [0,7,0,0,8,0,6,1,5],
              [2,5,3,1,0,4,0,0,0],
              [6,0,0,0,9,5,0,2,0],
              [0,6,7,0,0,0,0,0,1],
              [0,0,5,0,0,8,7,6,4],
              [3,0,0,9,7,0,0,0,2],
              [0,9,0,4,5,7,0,0,0],
              [5,8,2,0,3,0,9,0,0],
              [0,0,0,0,0,0,1,5,6]
])

In [457]:
def extract_sub_matrix(sudoku_mat: np.ndarray, n: int) -> np.ndarray:
    assert 0 <= n <= 8, 'Submatrix number must be betwenn 0 and 8'
    if n == 0:
        return sudoku_mat[0:3,0:3]
    elif n == 1:
        return sudoku_mat[3:6,0:3]
    elif n == 2:
        return sudoku_mat[6:9,0:3]
    elif n == 3:
        return sudoku_mat[0:3,3:6]
    elif n == 4:
        return sudoku_mat[3:6,3:6]
    elif n == 5:
        return sudoku_mat[6:9,3:6]
    elif n == 6:
        return sudoku_mat[0:3,6:9]
    elif n == 7:
        return sudoku_mat[3:6,6:9]
    elif n == 8:
        return sudoku_mat[6:9,6:9]
    
def check_validity_sub_matrix(sudoku_mat: np.ndarray, n: int) -> bool:
    is_valid = True
    sub_matrix = extract_sub_matrix(sudoku_mat,n)
    unique_values, counts = np.unique(sub_matrix.flatten(), return_counts=True)
    for value, count in zip(unique_values, counts):
        if value != 0 and count > 1:
            is_valid = False
    return is_valid

def extract_row(sudoku_mat: np.ndarray, n: int) -> np.ndarray:
    assert 0 <= n <= 8, 'Row number must be betwenn 0 and 8'
    return sudoku_mat[n,:]

def check_validity_row(sudoku_mat: np.ndarray, n: int) -> bool:
    is_valid = True
    row = extract_row(sudoku_mat,n)
    unique_values, counts = np.unique(row.flatten(), return_counts=True)
    for value, count in zip(unique_values, counts):
        if value != 0 and count > 1:
            is_valid = False
    return is_valid

def extract_column(sudoku_mat: np.ndarray, n: int) -> np.ndarray:
    assert 0 <= n <= 8, 'Column number must be betwenn 0 and 8'
    return sudoku_mat[:,n]

def check_validity_column(sudoku_mat: np.ndarray, n: int) -> bool:
    is_valid = True
    column = extract_column(sudoku_mat,n)
    unique_values, counts = np.unique(column.flatten(), return_counts=True)
    for value, count in zip(unique_values, counts):
        if value != 0 and count > 1:
            is_valid = False
    return is_valid

def check_validity_puzzle(sudoku_mat: np.ndarray) -> bool:
    is_valid = True
    for n in range(9):
        if not (check_validity_column(sudoku_mat,n) 
                and check_validity_row(sudoku_mat,n) 
                and check_validity_sub_matrix(sudoku_mat, n)
               ):
            is_valid = False
    return is_valid

def generate_n_possibility(sudoku_mat: np.ndarray, n: int) -> np.ndarray:
    assert 1 <= n <= 9, 'Number to test must be between 1 and 9'
    possibility_matrix = np.full((9, 9), n)
    for m in range(9):
        if n in extract_sub_matrix(sudoku_mat, m):
            possibility_sub_matrix = extract_sub_matrix(possibility_matrix,m)
            possibility_sub_matrix[:,:] = 0
        if n in extract_row(sudoku_mat, m):
            possibility_row = extract_row(possibility_matrix,m)
            possibility_row[:] = 0
        if n in extract_column(sudoku_mat, m):
            possibility_column = extract_column(possibility_matrix,m)
            possibility_column[:] = 0
        
    possibility_matrix[sudoku_mat!=0] = 0
    
    return possibility_matrix

def generate_possibility_tensor(sudoku_mat) -> np.ndarray:
    n_possibilities = []
    for n in range(1,10):
        n_possibilities.append(generate_n_possibility(sudoku_mat,n))
    possibility_tensor = np.empty((9,9),dtype=object)
    for i in range(9):
        for j in range(9):
            for n in range(9):
                possibility_matrix =  n_possibilities[n]
                current_value = possibility_matrix[i,j]
                if current_value != 0:
                    try:
                        possibility_tensor[i,j].append(current_value)
                    except:
                        possibility_tensor[i,j] = [current_value]
    return possibility_tensor

def identify_unique_possibilities(sudoku_mat: np.ndarray) -> list:
    possibility_tensor = generate_possibility_tensor(sudoku_mat)
    cells_to_fill = []
    for i in range(9):
        for j in range(9):
            try:
                num_options = len(possibility_tensor[i,j])
            except:
                num_options = 0
            if num_options == 1:
                cells_to_fill.append([possibility_tensor[i,j][0],i,j])
    return cells_to_fill

def is_puzzle_complete(sudoku_mat: np.ndarray) -> bool:
    return not 0 in sudoku_mat
                
def update_matrix_cells(sudoku_mat: np.ndarray, cells_to_fill: list) -> np.ndarray:
    for changes in cells_to_fill:
        n,i,j = changes
        sudoku_mat[i,j] = n
    return sudoku_mat

def identify_lone_posibilities(sudoku_mat: np.ndarray):
    cells_to_fill = []
    for n in range(1,10):
        poss_n = generate_n_possibility(sudoku_mat,n)
        for m in range(9):
            if (extract_column(poss_n,m) == n).sum() == 1:
                i = np.where(extract_column(poss_n,m) == n)[0][0]
                cells_to_fill.append([n,i,m])
            if (extract_row(poss_n,m) == n).sum() == 1:
                j = np.where(extract_row(poss_n,m) == n)[0][0]
                cells_to_fill.append([n,m,j])
            if (extract_sub_matrix(poss_n,m) == n).sum() == 1:
                pos = np.where(extract_sub_matrix(poss_n,m) == n)
                i,j = pos[0][0],  pos[1][0]
                if m==1:
                    i += 3
                elif m==2:
                    i += 6
                elif m==3:
                    j+=3
                elif m==4:
                    i+=3
                    j+=3
                elif m==5:
                    i+=6
                    j+=3
                elif m==6:
                    j+=6
                elif m==7:
                    i+=3
                    j+=6
                elif m==8:
                    i+=6
                    j+=6
                cells_to_fill.append([n,i,j])
    return cells_to_fill

def solve_puzzle(sudoku_mat: np.ndarray) -> np.ndarray:
    
    while (not is_puzzle_complete(sudoku_mat)) and (check_validity_puzzle(sudoku_mat)):
        
        current_sudoku = sudoku_mat.copy()
        cells_to_fill_lone = identify_lone_posibilities(sudoku_mat)
        cells_to_fill_unique = identify_unique_possibilities(sudoku_mat)
        cells_to_fill = cells_to_fill_unique + cells_to_fill_lone
        
        if cells_to_fill:
            update_matrix_cells(sudoku_mat,cells_to_fill)
        else:
            print('Puzzle not finished, insufficient info')
            break
    if is_puzzle_complete(sudoku_mat) and check_validity_puzzle(sudoku_mat):
        print('Puzzle is complete!')
    elif not check_validity_puzzle(sudoku_mat):
        print('Puzzle is incorrect')
    return sudoku_mat

In [666]:
sudoku_mat = np.array([
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
])

In [667]:
solve_puzzle(sudoku_mat)

Puzzle not finished, insufficient info


array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [668]:
original_version = sudoku_mat.copy()

def iter_solution_generation(sudoku_mat,original_version):
    if is_puzzle_complete(sudoku_mat) and check_validity_puzzle(sudoku_mat):
        print('Puzzle Complete')
        print(sudoku_mat)
        return sudoku_mat
    elif not check_validity_puzzle(sudoku_mat):
        print('Puzzle Error, Restart')
        sudoku_mat = original_version.copy()
        iter_solution_generation(sudoku_mat,original_version)
    else:
        poss = generate_possibility_tensor(sudoku_mat)
        non_null_options = np.where(poss)
        num_options = len(non_null_options[0])
        rd = random.randint(0, num_options-1)
        i = non_null_options[0][rd]
        j = non_null_options[1][rd]
        print(i,j,poss[i,j])
        random_solution = random.choice(poss[i,j])
        sudoku_mat[i,j] = random_solution
        solve_puzzle(sudoku_mat)
        return iter_solution_generation(sudoku_mat,original_version)
        

In [669]:
original_version = sudoku_mat.copy()

In [670]:
iter_solution_generation(sudoku_mat,original_version)

8 0 [1, 2, 3, 4, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
7 8 [1, 2, 3, 4, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
2 2 [1, 2, 3, 4, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
1 3 [1, 2, 3, 4, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
1 4 [1, 2, 4, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
1 6 [2, 4, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
6 4 [2, 3, 4, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
6 0 [1, 2, 3, 4, 6, 7, 9]
Puzzle not finished, insufficient info
7 7 [1, 2, 3, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
5 0 [1, 2, 3, 4, 5, 7, 9]
Puzzle not finished, insufficient info
0 0 [2, 4, 5, 7, 9]
Puzzle not finished, insufficient info
0 6 [1, 2, 3, 4, 6, 7, 9]
Puzzle not finished, insufficient info
7 4 [3, 6, 7, 8, 9]
Puzzle not finished, insufficient info
2 1 [2, 3, 4, 6, 7, 8, 9]
Puzzle not finished, insufficient info
2 5 [4, 5, 6, 7, 8, 9]
Puzzle not finished, insufficient info
5 1 [1,